# Bayesian Optimization

We implement a Bayesian optimization for LED configurations that most closely match a target
light spectrum. We will work with a helper class, `SelfDrivingLabDemo` for setting up the
spectrophotometer sensor, measuring data from the sensor,
generating random inputs, and measuring the objective function (i.e. mean absolute
error). From there, we perform an experiment with 100 iterations, visualize the
results, and compare against random search. See
[2.0-random-search.ipynb](2.0-random-search.ipynb) for some links on troubleshooting
installation setup and a linear version of running an optimization campaign without the
`SelfDrivingLabDemo` class.

## Setup

Package installation and imports

### Package Installation

Assuming you're running this notebook inside of the GitHub repository, do a local
installation (`-e`) in the directory one level about this `notebooks` directory (`../.`)
as follows.
> Note: This only needs to be run once.

In [ ]:
%pip install -e ../.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2 # just some IPython magic to recognize changes to installed packages
import pandas as pd
from self_driving_lab_demo.core import SelfDrivingLabDemo

### SelfDrivingLabDemo

We'll instantiate the class and verify some of the functionality described in the random
search tutorial ([`2.0-random-search.ipynb`](2.0-random-search.ipynb)).

#### Instantiation

Now, we instantiate the `SelfDrivingLabDemo` class with `autoload=True` so that records
a target to optimize against. This involves selecting a set of target measurements as the "true" input values (i.e. the input
brightness and RGB values that define the target spectrum) based on a random seed,
setting the LED to those values, and then recording the spectrum intensities.

> Note: Instantiating with autoload=True will light the LED briefly.

In [2]:
sdl = SelfDrivingLabDemo(autoload=True)

#### Functionality

We can do similar things to what was done in `2.0-random-search.ipynb`. For example, getting
random inputs, observing the sensor data, and evaluating the objective function.

In [3]:
[sdl.get_random_inputs(), sdl.get_random_inputs()]

[(0.38697802427798167, 111, 218, 177), (0.047088673943824766, 248, 194, 200)]

In [4]:
sdl.observe_sensor_data(*sdl.get_random_inputs())

(2675, 637, 670, 0, 941, 378, 670, 697, 607, 0)

In [5]:
sdl.evaluate(*sdl.get_random_inputs())

{'ch415_violet': 2581,
 'ch445_indigo': 5205,
 'ch480_blue': 6325,
 'ch515_cyan': 56242,
 'ch560_green': 2580,
 'ch615_yellow': 4863,
 'ch670_orange': 6323,
 'ch720_red': 0,
 'ch_clear': 0,
 'ch_nir': 0,
 'mae': 19639.4,
 'rmse': 32372.39752937678}

## Optimization

While there are great numerical tutorials comparing [grid search vs. random search vs.
Bayesian optimization](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46), here, we'll compare these three search methods in a way that perhaps you've never seen before,
namely a self-driving laboratory demo!

### Setup

We define our optimization task parameters and take care of imports.

### Optimization Task Parameters

We'll use 81 iterations repeated 10 times. The use of 81 iterations instead of something
"cleaner" like 50 or 100 is due to constraints of doing uniform (full-factorial) grid
search. $n^d$ number of points are required for uniform grid search, where $n$ and $d$
represent number of points per dimension (`n_pts_per_dim`) and number of dimensions
(`4`), respectively.

In [6]:
num_iter = 81
num_repeats = 10
SEEDS = range(10, 10 + num_repeats)

We also instantiate multiple `SelfDrivingLabDemo` instances, each with their own
unique target spectrum.

In [7]:
sdls = [SelfDrivingLabDemo(autoload=True, target_seed=seed) for seed in SEEDS]

Notice that the target_data is different for each.

In [8]:
pd.DataFrame([sdl.target_data for sdl in sdls], columns=sdl.channel_names)

,ch415_violet,ch445_indigo,ch480_blue,ch515_cyan,ch560_green,ch615_yellow,ch670_orange,ch720_red,ch_clear,ch_nir
0,8779,4517,26550,59658,6955,4683,17180,0,0,0
1,1651,390,955,0,203,65535,955,69,0,0
2,4343,5167,19947,685,14112,4386,956,0,0,0
3,65535,26162,63043,0,65535,35498,14601,0,0,0
4,38556,3204,65535,0,36510,3194,16210,65535,0,0
5,10558,5222,4326,10602,1429,65535,4327,0,0,0
6,11141,65535,1153,51321,2133,5309,1155,9298,0,0
7,4872,1713,56165,17209,2474,4685,15326,17195,0,0
8,908,65535,65535,0,1505,8001,2350,0,0,0
9,1433,3500,65535,0,2718,4115,2305,0,0,0




### Imports

We'll be using `scikit-learn`'s `ParameterGrid` for grid search, `self_driving_lab_demo`'s built-in
`get_random_inputs` for random search, and `ax-platform`'s Gaussian Process Expected
Improvement (GPEI) model for Bayesian
optimization. To help with defining the grid search space, we will also use the
`bounds` and `parameters` class property of `SelfDrivingLabDemo` for convenience.

In [9]:
import numpy as np
from tqdm import trange, tqdm
from sklearn.model_selection import ParameterGrid
from ax import optimize

In [10]:
sdls[0].bounds

{'brightness': [0.0, 0.5], 'R': [0, 255], 'G': [0, 255], 'B': [0, 255]}

In [11]:
sdls[0].parameters

[{'name': 'brightness', 'type': 'range', 'bounds': [0.0, 0.5]},
 {'name': 'R', 'type': 'range', 'bounds': [0, 255]},
 {'name': 'G', 'type': 'range', 'bounds': [0, 255]},
 {'name': 'B', 'type': 'range', 'bounds': [0, 255]}]

### Grid Search

First, we need to define our parameter grid. We'll divide up the 4-dimensional parameter
space as evenly as possible (see `num_pts_per_dim` below).

In [12]:
param_grid = {}
num_pts_per_dim = round(num_iter ** (1 / len(sdl.bounds)))
for name, bnd in sdl.bounds.items():
    param_grid[name] = np.linspace(bnd[0], bnd[1], num=num_pts_per_dim)
    if isinstance(bnd[0], int):
        param_grid[name] = np.round(param_grid[name]).astype(int)
print(f"num_pts_per_dim: {num_pts_per_dim}")

num_pts_per_dim: 3


Notice that there are only 3 distinct values along each dimension.

In [13]:
param_grid

{'brightness': array([0.  , 0.25, 0.5 ]),
 'R': array([  0, 128, 255]),
 'G': array([  0, 128, 255]),
 'B': array([  0, 128, 255])}

After assembling the full grid, notice that the total number of points is $3^4 = 81$.

In [14]:
grid = list(ParameterGrid(param_grid))
print("grid:\n", grid[0:4], "...", grid[-1:])
print("\nNumber of grid points: ", len(grid))

grid:
 [{'B': 0, 'G': 0, 'R': 0, 'brightness': 0.0}, {'B': 0, 'G': 0, 'R': 0, 'brightness': 0.25}, {'B': 0, 'G': 0, 'R': 0, 'brightness': 0.5}, {'B': 0, 'G': 0, 'R': 128, 'brightness': 0.0}] ... [{'B': 255, 'G': 255, 'R': 255, 'brightness': 0.5}]

Number of grid points:  81


Now, we can start the actual search. The grid search locations are fixed
for each of the repeat optimization campaigns; however the observed sensor data will be
stochastic and the target spectrum is different for each repeat run. An alternative approach to setting a
fixed budget and varying the target solution would be to see how many iterations it takes to meet a criteria for the
objective function similar to [this post](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46); however, a fixed budget seems more characteristic of a real chemistry
or materials optimization campaign due to limits on funding, time, and other resources:
(i.e. we'll search until we find what we're looking for, until we run out of
resources, or until we decide it's no longer worth the expense, whichever comes first).

In [15]:
grid_data = [
    [
        sdl.evaluate(pt["brightness"], pt["R"], pt["G"], pt["B"])
        for pt in grid
    ]
    for sdl in tqdm(sdls)
]

100%|██████████| 10/10 [11:21<00:00, 68.11s/it]


### Random Search

Now, let's perform random search as we did before in
[`2.0-random-search.ipynb`](2.0-random-search.ipynb), storing the inputs and outputs as we go.

In [82]:
%%time
random_inputs = []
random_data = []
for _ in tqdm(range(num_repeats)):
    ri = []
    od = []
    for i in range(num_iter):
        ri.append(sdl.get_random_inputs())
        od.append(sdl.evaluate(*ri[i]))
    random_inputs.append(ri)
    random_data.append(od)

100%|██████████| 10/10 [11:20<00:00, 68.06s/it]

CPU times: user 1min 20s, sys: 49.2 s, total: 2min 9s
Wall time: 11min 20s


### Bayesian Optimization

Ax may run out of memory for low-RAM RPi's such as RPi Zero 2. Even on RPi 4B/RPi
400, it will be slow* for more than about 50 iterations due to the lack of MKL
optimization (a feature of RPi compute hardware).

\*i.e. maybe slower than a demo/tutorial should be

In [21]:
%%time
best_parameters = []
values = []
experiment = []
model = []

for sdl in tqdm(sdls):
    def evaluation_function(parameters):
        data = sdl.evaluate(
            brightness=parameters["brightness"],
            R=parameters["R"],
            G=parameters["G"],
            B=parameters["B"],
        )
        return data["mae"]

    bp, v, exp, m = optimize(
        parameters=sdl.parameters,
        evaluation_function=evaluation_function,
        minimize=True,
        total_trials = num_iter,
    )

    best_parameters.append(bp)
    values.append(v)
    experiment.append(exp)
    model.append(m)

[INFO 08-19 11:13:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter brightness. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-19 11:13:40] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-19 11:13:40] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter G. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-19 11:13:40] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-19 1

CPU times: user 8h 13min 36s, sys: 18min 47s, total: 8h 32min 24s
Wall time: 3h 23min 24s


### Visualization

Now that we've run our three optimizations, let's compare the performance in tabular
form and visually.

### Preparing the data

In [91]:
grid_mae = [[g["mae"] for g in gd] for gd in grid_data]
random_mae = [[r["mae"] for r in rd] for rd in random_data]
bayesian_mae = [
    [trial.objective_mean for trial in exp.trials.values()]
    for exp in experiment
]

In [97]:
mae = np.array([grid_mae, random_mae, bayesian_mae])
mae.shape

(3, 10, 81)

### Tabular

> SPOILER: the following cell contains the bug!

In [158]:
avg_mae = np.minimum.accumulate(np.mean(mae, axis=1), axis=1)
std_mae = np.std(mae, axis=1)
avg_mae.shape

(3, 81)

In [184]:
np.mean(random_mae)

14430.651851851851

In [159]:
best_avg_mae = np.min(avg_mae, axis=1)
best_avg_mae

array([11818.8 , 10281.38, 11805.9 ])

### Best Objective vs. Iteration

In [160]:
names = ["grid", "random", "bayesian"]
df = pd.DataFrame({
    **{f"{n}_mae": m for n, m in zip(names, avg_mae)},
    **{f"{n}_std": s for n, s in zip(names, std_mae)},
})


In [161]:
mae_df = pd.melt(df.reset_index(), id_vars=["index"], value_vars = ["grid_mae", "random_mae", "bayesian_mae"], var_name="method", value_name="mae")

std_df = pd.melt(df.reset_index(), id_vars=["index"], value_vars = ["grid_std", "random_std", "bayesian_std"], var_name="method", value_name="std")

mae_df.loc[:, "method"] = mae_df.loc[:, "method"].apply(lambda x: x.replace("_mae", ""))
std_df.loc[:, "method"] = std_df.loc[:, "method"].apply(lambda x: x.replace("_std", ""))

In [162]:
results_df = mae_df.merge(std_df, on=["method", "index"]).rename(columns=dict(index="iteration"))
results_df

,iteration,method,mae,std
0,0,grid,16398.42,6658.779231
1,1,grid,13822.91,6191.477818
2,2,grid,13822.91,5918.128604
3,3,grid,13413.53,6162.685097
4,4,grid,13413.53,9426.585838
...,...,...,...,...
238,76,bayesian,11805.90,8111.040048
239,77,bayesian,11805.90,6233.286459
240,78,bayesian,11805.90,6054.450131
241,79,bayesian,11805.90,7748.289149


In [164]:
import plotly.express as px
fig = px.line(results_df,
        x="iteration",
        y="mae",
        # error_y="std",
        color="method"
        )
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'method=grid<br>iteration=%{x}<br>mae=%{y}<extra></extra>',
              'legendgroup': 'grid',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'grid',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
                          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
                          36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
                          54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
                          72, 73, 74, 75, 76, 77, 78, 79, 80]),
              'xaxis': 'x',
              'y': array([16398.42, 13822.91, 13822.91, 13413.53, 13413.53, 13413.53, 13413.53,
                          13413.53, 13413.53, 13413.53, 13413.53, 13413.53, 13413.53, 13413.53,
                          13413.53, 13413.53, 12856.92, 12856.92, 12856.92, 12856.92, 12856.92,
                          12856.92, 12856.92, 12856.92, 12856.92, 12824.56, 12824.56, 12824.56,
                          12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56,
                          12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56,
                          12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56,
                          12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56, 12824.56,
                          12824.56, 12582.52, 12582.52, 12582.52, 12582.52, 12582.52, 12582.52,
                          12582.52, 12582.52, 12582.52, 12582.52, 12582.52, 12582.52, 12582.52,
                          11818.8 , 11818.8 , 11818.8 , 11818.8 , 11818.8 , 11818.8 , 11818.8 ,
                          11818.8 , 11818.8 , 11818.8 , 11818.8 ]),
              'yaxis': 'y'},
             {'hovertemplate': 'method=random<br>iteration=%{x}<br>mae=%{y}<extra></extra>',
              'legendgroup': 'random',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'random',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
                          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
                          36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
                          54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
                          72, 73, 74, 75, 76, 77, 78, 79, 80]),
              'xaxis': 'x',
              'y': array([13632.98, 13632.98, 13632.98, 13632.98, 13246.11, 13246.11, 13246.11,
                          13246.11, 13246.11, 13246.11, 12310.14, 12310.14, 12310.14, 12310.14,
                          12270.21, 12270.21, 12270.21, 12270.21, 11705.6 , 11705.6 , 11705.6 ,
                          11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 ,
                          11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 ,
                          11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 , 11705.6 , 10281.38,
                          10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38,
                          10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38,
                          10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38,
                          10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38,
                          10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38, 10281.38,
              

### Example Output

![blooper](bayesian-optimization-blooper.png)

In [172]:
sdls[0].get_target_inputs()

(0.4780008548144877, 52, 211, 38)

In [176]:
sdls[0].target_data

(8779, 4517, 26550, 59658, 6955, 4683, 17180, 0, 0, 0)

In [188]:
check_data = []
from time import sleep
for _ in range(10):
    sleep(1.0)
    check_data.append(sdls[0].evaluate(*sdls[0].get_target_inputs()))

In [190]:
pd.DataFrame(check_data).agg([np.mean, np.std]).T

,mean,std
ch415_violet,17928.500000,25288.711738
ch445_indigo,8746.200000,9379.586248
ch480_blue,19396.100000,17832.023101
ch515_cyan,4870.900000,12126.813252
ch560_green,11908.300000,18944.797128
ch615_yellow,18406.400000,19626.981147
ch670_orange,30584.100000,20868.707815
ch720_red,5776.100000,9066.354228
ch_clear,1294.100000,2017.116669
ch_nir,0.000000,0.000000


In [196]:
sdl = SelfDrivingLabDemo(autoload=True, target_seed=15)

single_check_data = []
from time import sleep
for _ in range(10):
    sleep(1.0)
    single_check_data.append(sdl.evaluate(*sdl.get_target_inputs()))

In [197]:
pd.DataFrame(single_check_data).agg([np.mean, np.std]).T

,mean,std
ch415_violet,17007.200000,25480.302226
ch445_indigo,19130.100000,25168.441154
ch480_blue,8367.500000,6743.886042
ch515_cyan,4314.800000,6810.185734
ch560_green,5255.300000,5366.718034
ch615_yellow,8263.600000,8783.939956
ch670_orange,9396.500000,12130.862214
ch720_red,8446.000000,20422.496588
ch_clear,12985.800000,27127.159317
ch_nir,0.000000,0.000000


## Up Next

[`2.2-sensor-simulator.ipynb`](2.2-sensor-simulator.ipynb)
> 🕵️ Time to troubleshoot! Running simulations can help us to troubleshoot the source
> of the discrepancy. SPOILER: Oh! It was an issue with data processing 🤦 (but was that
> all? 🤨)

## Code Graveyard

In [ ]:
# for _ in trange(num_repeats):
#     for params in tqdm(grid):
#         sdl.evaluate(*params)

# pd.concat((mae_df, std_df), axis=1, join="inner")